## Example: Heat Engines

Endoreversible thermodynamics is subfield of irreversible thermodynamics focusing on the fundamental analysis of energy conversion accounting for finite resistances to heat transfer. Classical thermodynamics studies the theoretical limit of engines without the heat transfer resistances that limit performance in real world applications. It has been found that the efficiency bounds developed in this manner are generally closer estimates of actual performance than the more familiar Carnot efficiency described in introductory textbooks. The example described here is generally attributed to Curzon and Ahlborn (1975).

>Curzon, F. L., & Ahlborn, B. (1975). Efficiency of a Carnot engine at maximum power output. American Journal of Physics, 43(1), 22-24. 

The following paper applies these concepts to the optimal design of a geothermally powered air conditioning system. This paper would provide a good didactic example for the solution of a bilinear optimization problem.

> Davis, G. W., & Wu, C. (1997). Finite time analysis of a geothermal heat engine driven air conditioning system. Energy conversion and management, 38(3), 263-268.

Given a hot and cold reservoir at temperature $T_h$ and $T_c$, respectively, the mechanical power $\dot{w}$ that is extracted by any machine is subject to the energy balances.

$$
\begin{align*}
\dot{w} & = q_h - q_c & \text{power production}\\
\dot{q}_h & = U_h (T_h - T_h') & \text{heat transfer - hot} \\
\dot{q}_c & = U_c (T_c' - T_c) & \text{heat transfer - cold}\\
\end{align*}
$$

The heat transfer coefficients $U_h$ and $U_c$ model the transmission of heat to and from the engine that is converting heat to work. Heat is released from the hot reservoir at temperature $T_h'$ thereby increasing increasing entropy of the surroundings at a rate ${\dot{q}_h}/{T_h'}$. The heat absorbed at the cold temperature reservoir is decreasing entropy of the surroundings at a rate ${\dot{q}_c}/{T_c'}$. From the second law,

$$
\begin{align*}
\frac{\dot{q}_h}{T_h'} < \frac{\dot{q}_c}{T_c'} \\
\end{align*}
$$

Defining $\dot{\sigma}_h = \dot{q}_h/T_h'$ and $\dot{\sigma}_c = \dot{q}_c/T_c'$, and given data for the hot and cold utilities, $T_h$, $T_c$, $U_h$ and $U_c$, the problem is find the maximum mechanical power $\dot{w}$ subject to the constraints

$$
\begin{align*}
& \max\ \dot{w} \\
\\
\text{s.t.}\qquad\qquad
\dot{w} & = \dot{q}_h - \dot{q}_c & \text{power production}\\
\dot{q}_h & = U_h (T_h - T_h') & \text{heat transfer - hot} \\
\dot{q}_c & = U_c (T_c' - T_c) & \text{heat transfer - cold}\\
\dot{q}_h & = \dot{\sigma}_h T_h' \\
\dot{q}_c & = \dot{\sigma}_c T_c' \\
\dot{\sigma}_h & < \dot{\sigma}_c &
\end{align*}
$$

The notation $\dot{\sigma}$ is used in place of entropy in order to formulate the model with non-negative variables.

In [ ]:
import pyomo.kernel as pmo

Th = 500
Tc = 300

Uh = 3
Uc = 3

m = pmo.block()

m.Th = pmo.variable(lb=0)
m.Tc = pmo.variable(lb=0)
m.qh = pmo.variable(lb=0)
m.qc = pmo.variable(lb=0)
m.sh = pmo.variable(lb=0)
m.sc = pmo.variable(lb=0)
m.w = pmo.variable()

m.energy_balance = pmo.constraint(m.w == m.qh - m.qc)
m.heat_source = pmo.constraint(m.qh == Uh*(Th - m.Th))
m.heat_sink = pmo.constraint(m.qc == Uc*(m.Tc - Tc))
m.entropy_out = pmo.constraint(m.qh == m.sh * m.Th)
m.entropy_in = pmo.constraint(m.qc == m.sc * m.Tc)
m.second_law = pmo.constraint(m.sh <= m.sc)

m.objective = pmo.objective(-m.w)

pmo.SolverFactory('ipopt').solve(m)

print(f"maximum work = {m.w()}")

print("Th = ", m.Th())
print("Tc = ", m.Tc())
print("qh = ", m.qh())
print("qc = ", m.qc())
print("sh = ", m.sh())
print("sc = ", m.sc())
